In [1]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
from scipy.special import gamma


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
os.chdir("../data")

In [3]:
recommendee = "Fro116"
neighborhood_size = 64
confidence_interval = 0.99
full_neighborhoods = False
perform_regression = True

In [4]:
anime = pd.read_csv("AnimeList.csv")
anime = anime[["anime_id", "title", "type"]]

In [5]:
df = pd.read_csv("UserAnimeList.csv")

In [6]:
len(df["username"].unique()), len(df["anime_id"].unique()), len(df) / (
    len(df["username"].unique()) * len(df["anime_id"].unique())
), len(df)

(283045, 14478, 0.01954064606703893, 80076112)

In [7]:
filtered_df = df[["username", "anime_id", "my_score"]].loc[lambda x: x["my_score"] != 0]

In [8]:
# add additional user anime-lists
extraUsers = pickle.load(open("user_profiles/ExtraUserAnimeLists.pkl", "rb"))
filtered_df = filtered_df.loc[lambda x: ~x["username"].isin(extraUsers.username)]
filtered_df = pd.concat([filtered_df, extraUsers], ignore_index=True)

In [9]:
average_rating = filtered_df["my_score"].mean()
user_bias = (
    pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
        {"my_score": "user_bias"}, axis=1
    )
    - average_rating
)
anime_bias = (
    pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
        {"my_score": "anime_bias"}, axis=1
    )
    - average_rating
)

In [10]:
filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
    user_bias, on=["username"]
)
filtered_df["normalized_score"] = (
    filtered_df["my_score"]
    - filtered_df["anime_bias"]
    - filtered_df["user_bias"]
    - average_rating
)
filtered_df = filtered_df.set_index("username")
filtered_df = filtered_df.dropna()

In [11]:
filtered_df

,anime_id,my_score,anime_bias,user_bias,normalized_score
username,,,,,
karthiga,21,9,0.960563,-0.059899,0.605474
karthiga,59,7,0.040202,-0.059899,-0.474165
karthiga,74,7,0.316282,-0.059899,-0.750244
karthiga,120,7,0.309858,-0.059899,-0.743820
karthiga,178,7,-0.227339,-0.059899,-0.206623
...,...,...,...,...,...
temptemptemp,10040,6,-1.636718,-1.493861,1.636718
cinnamoroller,12963,10,-0.798861,2.506139,0.798861
inactiveX,5143,7,-0.652952,-0.493861,0.652952


In [12]:
all_corrs = pickle.load(open("item_correlations/correlations.pkl", "rb"))

In [13]:
all_corrs["similarity"] = all_corrs["corr"].abs()
all_corrs = all_corrs.dropna()

In [14]:
corrs = all_corrs.loc[
    lambda x: x.index.get_level_values("anime_id_x")
    != x.index.get_level_values("anime_id_y")
]

In [15]:
if full_neighborhoods:
    corrs = corrs.groupby("anime_id_x").apply(lambda x: x.sort_values(by="similarity"))
else:
    corrs = corrs.groupby("anime_id_x").apply(
        lambda x: x.sort_values(by="similarity")[-neighborhood_size:]
    )

In [16]:
corrs.index = corrs.index.droplevel()

In [17]:
corrs

corr  corr_var   size  similarity
anime_id_x anime_id_y                                       
1          239         0.085037  0.000085  11599    0.085037
           440         0.085474  0.000131   7531    0.085474
           2236        0.088889  0.000030  33034    0.088889
           875         0.089690  0.000192   5127    0.089690
           790         0.089991  0.000040  24767    0.089991
...                         ...       ...    ...         ...
37908      28913       0.207284  0.915913      3    0.207284
           28915       0.226825  0.899748      3    0.226825
           28911       0.235301  0.892332      3    0.235301
           37124       0.267396  0.862111      3    0.267396
           37577       0.555375  0.478253      3    0.555375

[871837 rows x 4 columns]

In [18]:
score = filtered_df.loc[recommendee].merge(
    corrs.reset_index("anime_id_x"), left_on="anime_id", right_on="anime_id_y"
)

user_var = (
    pd.DataFrame(filtered_df.groupby("username")["normalized_score"].var())
    .rename({"normalized_score": "user_var"}, axis=1)
    .dropna()
)
score["user_var"] = user_var.loc[recommendee].squeeze()

anime_var = (
    pd.DataFrame(filtered_df.groupby("anime_id")["normalized_score"].var())
    .rename({"normalized_score": "anime_var"}, axis=1)
    .dropna()
)
score = score.merge(anime_var, on="anime_id")

score = score.drop("anime_id", axis=1).rename({"anime_id_x": "anime_id"}, axis=1)

if full_neighborhoods:
    score = (
        score.groupby("anime_id")
        .apply(lambda x: x.sort_values(by="similarity")[-neighborhood_size:])
        .reset_index(drop=True)
    )

In [19]:
score.head()

,my_score,anime_bias,user_bias,normalized_score,anime_id,corr,corr_var,size,similarity,user_var,anime_var
0,7,0.219361,-1.032528,0.319305,45,0.055748,0.000104,9546,0.055748,2.583767,1.630602
1,7,0.219361,-1.032528,0.319305,123,0.048346,0.000212,4696,0.048346,2.583767,1.630602
2,7,0.219361,-1.032528,0.319305,154,0.034332,0.000147,6792,0.034332,2.583767,1.630602
3,7,0.219361,-1.032528,0.319305,189,0.048751,0.000113,8816,0.048751,2.583767,1.630602
4,7,0.219361,-1.032528,0.319305,223,0.075094,0.000086,11440,0.075094,2.583767,1.630602


In [20]:
deltas = score.groupby("anime_id").apply(
    lambda x: np.dot(x["normalized_score"], x["corr"]) / x["corr"].abs().sum()
)
weights = score.groupby("anime_id").apply(lambda x: x["corr"].abs().sum())
counts = score.groupby("anime_id").size()

In [21]:
# The following formulae are used to compute the variance of the delta. Delta
# is a weighted sum of the form δ = Σ(s_i * w_i) / (Σw_i), where s_i is
# a vector scores for item i and w_i is the weight for item_i.
#
# By linearity, it suffices to compute (s_i * w_i) / (Σw_i). We assume that
# Var(s_i) is the same as the variance over the vector s_i (i.e. over
# all users s_i has rated). We treat w_i as a random variable with mean w_i
# and variance corr['corr_var']
#
# The variance for (w_i) / (Σw_i) can be estimated by doing a Taylor Approximation.
# See equation 20 of https://www.stat.cmu.edu/~hseltman/files/ratio.pdf. The
# formula for the ratio of two correlated variables R,S is
# Var(R/S) = E[R]^2/E[S]^2(Var[R]/E[R]^2 - 2Cov(R,S)/(E[R]E[S]) + Var[S]/E[S]^2)
#
# Lastly we take the product distribution of s_i and (w_i) / (Σw_i).
def correction_factor(x):
    return (
        1
        + x["corr_var"] / (x["corr"] ** 2)
        - 2 * x["corr_var"] / (x["corr"].abs().sum() * x["corr"].abs())
        + x["corr_var"].sum() / (x["corr"].abs().sum() ** 2)
    )


delta_var = score.groupby("anime_id").apply(
    lambda x: np.sum(x["user_var"] * x["corr"] ** 2 * correction_factor(x))
    / (x["corr"].abs().sum() ** 2)
)

# if the var < 0, then the ratio distribution approximation failed,
# usually because sample size is too small
delta_var.loc[lambda x: x < 0] = np.inf

# Apply a bessel correction to unbias the variance
average_weight = corrs.groupby("anime_id_x").apply(lambda x: x["corr"].abs().mean())
effective_sample_size = weights / average_weight
delta_var.loc[effective_sample_size <= 1] = np.inf
delta_var.loc[effective_sample_size > 1] *= effective_sample_size / (effective_sample_size - 1)

In [22]:
pred_df = pd.DataFrame()
pred_df["delta"] = deltas
pred_df["weight"] = weights
pred_df["counts"] = counts
pred_df["delta_sem"] = np.sqrt(delta_var)

# Unbias the standard deviation estimate
# See https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation#Results_for_the_normal_distribution
def standard_deviation_bias(n):
    if n < 1:
        return np.inf
    if gamma((n - 1) / 2) == np.inf:
        return 1
    return np.sqrt(2 / (n - 1)) * gamma(n / 2) / gamma((n - 1) / 2)
pred_df["delta_sem"] /= effective_sample_size.apply(standard_deviation_bias)

pred_df["blp"] = anime_bias + user_bias.loc[recommendee].squeeze() + average_rating
pred_df = pred_df.dropna()

recomendee_seen_shows = filtered_df.loc[recommendee].merge(pred_df, on=["anime_id"])
recomendee_seen_shows["target"] = (
    recomendee_seen_shows["my_score"] - recomendee_seen_shows["blp"]
)
if perform_regression:

    model = lm("target ~ delta + 0", recomendee_seen_shows)
    pred_df["score"] = model.predict(pred_df) + pred_df["blp"]
    pred_df["sem"] = np.sqrt(
        (
            (pred_df["delta_sem"] ** 2 + pred_df["delta"] ** 2)
            * (model.bse["delta"] ** 2 + model.params["delta"] ** 2)
        )
        - pred_df["delta"] ** 2 * model.params["delta"] ** 2
    )
else:
    pred_df["score"] = pred_df["delta"] + pred_df["blp"]
    pred_df["sem"] = pred_df["delta_sem"]


zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)
pred_df["score_lower_bound"] = pred_df["score"] - pred_df["sem"] * zscore
pred_df["score_upper_bound"] = pred_df["score"] + pred_df["sem"] * zscore

pred_df = pred_df.merge(anime, on="anime_id")
pred_df = pred_df.set_index("anime_id")

In [23]:
# confirm that setting blp = 1 is reasonable
print(lm("my_score ~ delta + blp + 0", recomendee_seen_shows).summary())

                                 OLS Regression Results                                
Dep. Variable:               my_score   R-squared (uncentered):                   0.954
Model:                            OLS   Adj. R-squared (uncentered):              0.954
Method:                 Least Squares   F-statistic:                              3406.
Date:                Wed, 05 May 2021   Prob (F-statistic):                   1.16e-219
Time:                        00:18:20   Log-Likelihood:                         -587.32
No. Observations:                 329   AIC:                                      1179.
Df Residuals:                     327   BIC:                                      1186.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [24]:
# confirm that the top shows are ones that the user rates highly
pred_df.loc[lambda x: x["delta"] > 0].sort_values(
    by="score_lower_bound", ascending=False
)[:20]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
34376,2.635013,1.384546,6,0.816029,7.087633,9.727624,0.860794,7.510365,11.944883,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
34375,2.844621,1.293363,5,0.856355,6.812681,9.662676,0.905679,7.329801,11.995552,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
3297,0.451652,5.067419,29,0.392683,7.645537,8.098043,0.397741,7.073530,9.122555,Aria The Origination,TV
34374,2.854099,1.311842,5,0.855545,6.490499,9.349990,0.905188,7.018380,11.681600,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no S...,Movie
1729,1.086106,1.245211,20,0.454452,7.033038,8.121195,0.469599,6.911588,9.330803,Maria-sama ga Miteru 3rd,OVA
397,1.430917,3.456904,22,0.566447,6.939673,8.373293,0.587074,6.861092,9.885494,Seikai no Senki II,TV
962,0.713016,4.994643,26,0.426111,7.256974,7.971337,0.434551,6.852007,9.090667,Aria The Natural,TV
4565,0.295710,1.577753,23,0.385004,7.528859,7.825128,0.388633,6.824076,8.826179,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,Movie
3371,1.094776,1.576788,13,0.538101,7.149197,8.246041,0.552119,6.823878,9.668204,Ginga Eiyuu Densetsu Gaiden,OVA


In [25]:
new_recs = pred_df.drop(filtered_df.loc[recommendee].anime_id, errors="ignore").loc[
    lambda x: (x["type"] != "Movie")
    & (x["type"] != "Special")
    & (x["type"] != "OVA")
    & (x["type"] != "ONA")
    & (x["type"] != "Music")
]

In [26]:
new_recs.loc[lambda x: (x["delta"] > 0)].sort_values(
    by="score_lower_bound", ascending=False
)[:20]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
444,1.058390,1.302614,22,0.428804,6.854022,7.914411,0.443848,6.771133,9.057688,Maria-sama ga Miteru: Haru,TV
3750,1.128924,0.919299,15,0.532463,7.006634,8.137691,0.547214,6.728162,9.547219,Maria-sama ga Miteru 4th,TV
153,0.424859,2.032601,28,0.331077,7.003295,7.428956,0.335846,6.563875,8.294038,Juuni Kokuki,TV
427,0.969124,0.641080,14,0.544314,6.938626,7.909581,0.556037,6.477324,9.341839,Kaleido Star,TV
18195,0.183485,1.013641,21,0.386246,7.255490,7.439322,0.389222,6.436752,8.441891,Little Busters!: Refrain,TV
158,1.051517,1.383636,25,0.391459,6.411603,7.465106,0.407212,6.416197,8.514016,Maria-sama ga Miteru,TV
2158,0.324454,1.504743,24,0.366292,7.028430,7.353497,0.370081,6.400231,8.306764,Terra e... (TV),TV
2164,0.016037,3.739247,37,0.278603,7.086931,7.102998,0.280456,6.380592,7.825404,Dennou Coil,TV
593,0.944226,1.553056,17,0.430226,6.550640,7.496650,0.442786,6.356110,8.637190,Mugen no Ryvius,TV
